# Champion Analysis: Thresh

In [1]:
from bs4 import BeautifulSoup as bs
from Champions import ChampionWikia

In [2]:
CHAMPION_NAME = 'Thresh'

In [3]:
CHAMPION = ChampionWikia(CHAMPION_NAME)
CHAMPION.get()

200

In [4]:
html = bs(CHAMPION.html, 'html.parser')

In [5]:
#Base Stats

for key, value in CHAMPION.base_stats.items():
    if html.find('span', id=key) is not None:
        CHAMPION.base_stats[key] = float(html.find('span', id=key).string)
        print(f'{key} --> {CHAMPION.base_stats[key]}')

for key, value in CHAMPION.base_stats_lvl.items():
    if html.find('span', id=key) is not None:
        CHAMPION.base_stats[key] = float(html.find('span', id=key).string)
        print(f'{key} --> {CHAMPION.base_stats[key]}')

Health_Thresh --> 561.0
HealthRegen_Thresh --> 7.0
ResourceBar_Thresh --> 274.0
ResourceRegen_Thresh --> 6.0
Armor_Thresh --> 28.0
AttackDamage_Thresh --> 56.0
MagicResist_Thresh --> 30.0
MovementSpeed_Thresh --> 335.0
AttackRange_Thresh --> 450.0
Health_Thresh_lvl --> 93.0
HealthRegen_Thresh_lvl --> 0.55
ResourceBar_Thresh_lvl --> 44.0
ResourceRegen_Thresh_lvl --> 0.8
Armor_Thresh_lvl --> 0.0
AttackDamage_Thresh_lvl --> 2.2
MagicResist_Thresh_lvl --> 0.5
